In [4]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
import joblib

kaggle_input="./kaggle/input"
# kaggle_working="./kaggle/working"
kaggle_temp="./kaggle/temp1"

pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 10)

In [5]:
f_train=pd.read_csv(f"{kaggle_temp}/f_train.csv")
# f_test=pd.read_csv(f"{kaggle_working}/f_test.csv")


In [6]:
t_sample=len(f_train)
f_train=f_train[:t_sample]
f_train

,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,...,RP_19.92savg55,RP_19.92savg56,RP_19.92savg57,RP_19.92savg58,RP_19.92savg59
0,1.0,0.0,0.000000,0.0,0.000000,...,0.132,0.166,0.128,0.136,0.212
1,1.0,0.0,0.000000,0.0,0.000000,...,0.132,0.166,0.128,0.136,0.212
2,1.0,0.0,0.000000,0.0,0.000000,...,0.132,0.166,0.128,0.136,0.212
3,1.0,0.0,0.000000,0.0,0.000000,...,0.132,0.166,0.128,0.136,0.212
4,1.0,0.0,0.000000,0.0,0.000000,...,0.132,0.166,0.128,0.136,0.212
...,...,...,...,...,...,...,...,...,...,...,...
11495,0.0,0.0,0.000000,0.0,0.000000,...,0.014,0.014,0.012,0.020,0.018
11496,0.0,0.0,0.166667,0.0,0.166667,...,0.014,0.014,0.012,0.020,0.018
11497,0.0,0.0,0.166667,0.0,0.166667,...,0.014,0.014,0.012,0.020,0.018
11498,0.0,0.0,0.166667,0.0,0.166667,...,0.014,0.014,0.012,0.020,0.018


In [7]:
f_train.fillna(method='bfill',inplace=True)
f_train.isna().sum()


seizure_vote      0
lpd_vote          0
gpd_vote          0
lrda_vote         0
grda_vote         0
                 ..
RP_19.92savg55    0
RP_19.92savg56    0
RP_19.92savg57    0
RP_19.92savg58    0
RP_19.92savg59    0
Length: 4806, dtype: int64

In [10]:
import time

f_train
vote_cols=['seizure_vote','lpd_vote','gpd_vote','lrda_vote','grda_vote','other_vote']
models=[RandomForestRegressor(n_estimators=100, random_state=3,n_jobs=-1, min_samples_split=2) for i in range(len(vote_cols))]
train_cols=[col for col in f_train.columns if col not in vote_cols]
for i in range(len(vote_cols)):
    start_time = time.time()
    
    print(f'training {vote_cols[i]} model')
    models[i].fit(f_train[train_cols],f_train[vote_cols[i]])
    joblib.dump(models[i],f'{kaggle_temp}/models/{vote_cols[i]}.joblib')
    print(f'trained and saved at {kaggle_temp}/models/{vote_cols[i]}.joblib')

    print("--- %s seconds --- for %s model" % (time.time() - start_time,vote_cols[i]))



training seizure_vote model


trained and saved at ./kaggle/temp1/models/seizure_vote.joblib
--- 39.84470796585083 seconds --- for seizure_vote model
training lpd_vote model
trained and saved at ./kaggle/temp1/models/lpd_vote.joblib
--- 24.934178113937378 seconds --- for lpd_vote model
training gpd_vote model
trained and saved at ./kaggle/temp1/models/gpd_vote.joblib
--- 31.21321702003479 seconds --- for gpd_vote model
training lrda_vote model
trained and saved at ./kaggle/temp1/models/lrda_vote.joblib
--- 27.210952520370483 seconds --- for lrda_vote model
training grda_vote model
trained and saved at ./kaggle/temp1/models/grda_vote.joblib
--- 25.216177225112915 seconds --- for grda_vote model
training other_vote model
trained and saved at ./kaggle/temp1/models/other_vote.joblib
--- 22.606230974197388 seconds --- for other_vote model


In [11]:
p_votes=pd.DataFrame(columns=vote_cols)
for i in range(len(models)):
    p_votes[vote_cols[i]]=models[i].predict(f_train[train_cols])

p_votes

,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
0,1.0,0.0,0.000000,0.0,0.000000,0.000000
1,1.0,0.0,0.000000,0.0,0.000000,0.000000
2,1.0,0.0,0.000000,0.0,0.000000,0.000000
3,1.0,0.0,0.000000,0.0,0.000000,0.000000
4,1.0,0.0,0.000000,0.0,0.000000,0.000000
...,...,...,...,...,...,...
11495,0.0,0.0,0.039918,0.0,0.039918,0.920164
11496,0.0,0.0,0.039918,0.0,0.039918,0.920164
11497,0.0,0.0,0.039918,0.0,0.039918,0.920164
11498,0.0,0.0,0.039918,0.0,0.039918,0.920164


In [12]:
# generalizing probs
for i in range(100):
   sum_votes=sum(p_votes.loc[i,vote_cols])
   p_votes.loc[i,vote_cols]=p_votes.loc[i,vote_cols].apply(lambda x: (x/sum_votes) )
p_votes.loc[30:31,:]

,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
30,0.707892,0.0,0.292108,0.0,0.0,0.0
31,0.707892,0.0,0.292108,0.0,0.0,0.0
